In [23]:
import pandas as pd
import numpy as np
import importlib

import data_file

df = pd.read_excel('data_files/salesgp.xlsx')

In [24]:
import sqlite3
conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"  # Replace 'tablename' with your table name
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [25]:
df_customers = df_customers[['sold_to_customer', 'sold_to_customer_n', 'customer_name', 
       'indirect_direct', 'channel', 'type', 
       'tier_new', 'tier']]

In [26]:
df_with_customers_types = pd.merge(df, df_customers, on = 'sold_to_customer', how='left')
df_with_customers_types = df_with_customers_types.dropna(subset='indirect_direct')

In [29]:
df_with_customers_types = df_with_customers_types[['company_code_n', 'customer_name', 'indirect_direct', 'channel', 'type', 'tier_new',
       'tier', 'bu', 'fiscal_year','sales_rsp_eur', 'sales_eur', 'cogs(ms)_eur', 'gp_eur']]

In [30]:
df_with_customers_types.loc[df_with_customers_types['tier'].isna(), 'tier'] = 'Direct'
df_with_customers_types.loc[df_with_customers_types['tier'] == 'None', 'tier'] = 'Direct'
df_with_customers_types.loc[df_with_customers_types['tier_new'].isna(), 'tier_new'] = 'Direct'
df_with_customers_types.loc[df_with_customers_types['tier_new'] == 'None', 'tier_new'] = 'Direct'

In [ ]:
grouped_df = df_with_customers_types.groupby(['company_code_n', 'tier_new', 'fiscal_year']).sum()[['sales_rsp_eur', 'sales_eur', 'cogs(ms)_eur', 'gp_eur']]

# Save the grouped dataframe to a new dataframe
result_df = grouped_df.reset_index()

# Write the dataframe to an Excel file
result_df.to_excel('data_files/sales_output.xlsx', index=False)